In [1]:
from dotenv import load_dotenv
import locale
import numpy as np
import os
import pandas as pd
import sqlalchemy as sa

pd.set_option("display.float_format", lambda val: locale.currency(val=val, symbol=True, grouping=True))
pd.set_option("display.max_rows", None)

engine: sa.Engine = sa.create_engine(os.getenv("URL_MYSQL"))

load_dotenv()

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")
locale.setlocale(locale.LC_MONETARY, "pt_BR.UTF-8")

In [ ]:
df_new: pd.DataFrame = pd.read_csv("../src/megasena.csv", encoding="utf-8-sig")

row_inserted: int = df_new.to_sql(name="megasena", con=engine, if_exists="append", index=False)
print(f"Foram {row_inserted} jogo(s) inserido(s) com sucesso!")

In [2]:
df_last: pd.DataFrame = pd.read_sql(sql=sa.text("SELECT concurso, data, bolas, acerto_6, rateio_6 FROM megasena"), con=engine)

df_last["concurso"] = df_last["concurso"].astype(str).str.zfill(4)
df_last["data"] = pd.to_datetime(df_last["data"]).dt.strftime("%x (%a)")
df_last["acerto_6"] = np.where(df_last["acerto_6"], "Alguém ganhou", "Ninguém ganhou")
df_last.columns = ["Concurso", "Data", "Bolas", "Acertou?", "Rateio"]

df_last.tail(15)

,Concurso,Data,Bolas,Acertou?,Rateio
2742,2743,29/06/2024 (Sáb),13 25 27 30 37 53,Ninguém ganhou,"0,00 R$"
2743,2744,02/07/2024 (Ter),10 25 26 33 34 38,Ninguém ganhou,"0,00 R$"
2744,2745,04/07/2024 (Qui),02 05 07 11 52 57,Alguém ganhou,"54.262.775,07 R$"
2745,2746,06/07/2024 (Sáb),22 27 30 43 51 56,Ninguém ganhou,"0,00 R$"
2746,2747,10/07/2024 (Qua),14 17 24 28 36 45,Ninguém ganhou,"0,00 R$"
2747,2748,13/07/2024 (Sáb),19 32 43 46 50 52,Ninguém ganhou,"0,00 R$"
2748,2749,16/07/2024 (Ter),08 25 27 38 43 44,Ninguém ganhou,"0,00 R$"
2749,2750,18/07/2024 (Qui),07 11 12 19 36 52,Ninguém ganhou,"0,00 R$"
2750,2751,20/07/2024 (Sáb),04 13 18 42 52 53,Ninguém ganhou,"0,00 R$"
2751,2752,23/07/2024 (Ter),04 15 24 40 44 47,Ninguém ganhou,"0,00 R$"


In [3]:
minhas_apostas: tuple = (
    "05 15 26 27 46 53",  # aposta n.° 1
    "03 12 19 20 45 47",  # aposta n.° 2
    "01 10 17 41 42 56",  # aposta n.° 3
    "02 10 13 27 53 55",  # aposta n.° 4
    "06 07 08 11 43 56",  # aposta n.° 5
    "08 10 14 25 33 34",  # aposta n.° 6
    "05 11 16 40 43 57",  # aposta n.° 7
    "04 05 08 13 17 38",  # aposta n.° 8
    "13 24 32 49 51 60",  # aposta n.° 9
    "11 16 19 43 58 60",  # aposta n.° 10
    "03 05 10 20 35 46",  # aposta n.° 11
    "02 09 10 19 31 57",  # aposta n.° 12
    "04 18 20 21 39 57",  # aposta n.° 13
    "02 11 22 36 49 60",  # aposta n.° 14
    "02 21 39 48 52 57",  # aposta n.° 15
    "14 41 45 50 54 59",  # aposta n.° 16
    "13 20 22 25 28 39",  # aposta n.° 17
    "01 16 21 34 49 54",  # aposta n.° 18
)

In [4]:
pd.read_sql(sql=sa.text(f"SELECT concurso, data, bolas, acerto_6, rateio_6 FROM megasena WHERE bolas IN {minhas_apostas}"), con=engine)

,concurso,data,bolas,acerto_6,rateio_6


In [9]:
for r in range(6, 3, -1):
    mega: dict[str: list] = {"Concurso": [], "Data": [], "Bolas": [], "Aposta n.°": []}

    stmt: str = f"SELECT concurso, data, bolas, acerto_{r}, rateio_{r} FROM megasena"

    for row in pd.read_sql(sql=sa.text(stmt), con=engine).itertuples(index=False, name=None):
        for aposta in minhas_apostas:
            bolas: list[str] = aposta.split()
            match: list[str] = []

            for x in range(6):
                if bolas[x] in row[2]:
                    match.append(bolas[x])

            if len(match) == r:
                mega["Concurso"].append(str(row[0]).zfill(4))
                mega["Data"].append(pd.to_datetime(row[1]).strftime("%x (%a)"))
                mega["Bolas"].append(" ".join(match))
                mega["Aposta n.°"].append(minhas_apostas.index(aposta) + 1)

    print(f"Lista de {r} acertos:")
    pd.DataFrame(mega) if len(pd.DataFrame(mega)) != 0 else print("Suas apostas não tiveram acertos...")

Lista de 6 acertos:
Suas apostas não tiveram acertos...
Lista de 5 acertos:
Lista de 4 acertos:


In [6]:
sua_aposta: str = "12 14 31 45 50 60"

mega: dict[str: list] = {"Concurso": [], "Data": [], "Bolas": [], "Acertos": []}

for row in pd.read_sql(sql=sa.text("SELECT * FROM megasena"), con=engine).itertuples(index=False, name=None):
    match: list[str] = []
    for aposta in sua_aposta.split():
        if aposta in row[2]:
            match.append(aposta)
    if len(match) >= 4:
        mega["Concurso"].append(str(row[0]).zfill(4))
        mega["Data"].append(pd.to_datetime(row[1]).strftime("%x (%a)"))
        mega["Bolas"].append(row[2])
        mega["Acertos"].append(len(match))

pd.DataFrame(mega)

,Concurso,Data,Bolas,Acertos
0,0571,12/06/2004 (Sáb),12 13 14 31 50 51,4
1,1277,23/04/2011 (Sáb),12 18 31 50 59 60,4


In [ ]:
stmt: str = """
    SELECT concurso, data, bolas, acerto_6, rateio_6
    FROM megasena
    WHERE data IN (
        SELECT MAX(data)
        FROM megasena
        GROUP BY YEAR(data)
        HAVING YEAR(data) <> YEAR(CURRENT_DATE)
    )
"""

df_mega_da_virada: pd.DataFrame = pd.read_sql(sql=sa.text(stmt), con=engine)

df_mega_da_virada["concurso"] = df_mega_da_virada["concurso"].astype(str).str.zfill(4)
df_mega_da_virada["data"] = pd.to_datetime(df_mega_da_virada["data"]).dt.strftime("%x (%a)")
df_mega_da_virada["acerto_6"] = np.where(df_mega_da_virada["acerto_6"].ne(0), "Alguém acertou", "Ninguém acertou")
df_mega_da_virada.columns = ["Concurso", "Data", "Bolas", "Acertou?", "Rateio"]

df_mega_da_virada

In [ ]:
apostas: dict[int: int] = {}

for x in minhas_apostas:
    y: list[str] = x.split()
    for z in range(len(y)):
        try:
            apostas[int(y[z])] += 1
        except KeyError:
            apostas[int(y[z])] = 1

apostas_ordenadas: dict[int: int] = {k: v for k, v in sorted(apostas.items(), key=lambda s: s[0])}

acertos: pd.DataFrame = pd.DataFrame(data=apostas_ordenadas, index=["acertos"], columns=apostas_ordenadas.keys())
acertos.index.name = "bolas"
acertos